In [1]:
import numpy as np
import cv2

In [15]:
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

In [16]:
my_faces = np.load('my_faces.npy')
unknown_faces = np.load('unknown_faces.npy')

In [17]:
import matplotlib.pyplot as plt

In [29]:
data = np.r_[my_faces.reshape(400,-1), unknown_faces.reshape(400,-1)]

In [30]:
# normalization
data = data / 255.0

In [31]:
data.shape

(800, 7500)

In [32]:
labels = np.zeros((data.shape[0],1))

In [33]:
labels[400:,:] = 1.0

In [34]:
np.unique(labels, return_counts=True)

(array([0., 1.]), array([400, 400], dtype=int64))

In [35]:
names = {
    0 : "Ravi",
    1 : "Unknown"
}

In [36]:
labels[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.],
       [0.]])

In [37]:
x_train, x_test, y_train, y_test = train_test_split(data, labels, test_size=0.25)

In [38]:
x_train.shape, x_test.shape, y_train.shape, y_test.shape

((600, 7500), (200, 7500), (600, 1), (200, 1))

In [39]:
y_train[:10]

array([[0.],
       [0.],
       [0.],
       [0.],
       [1.],
       [1.],
       [0.],
       [1.],
       [0.],
       [1.]])

In [40]:
logistic = LogisticRegression()
logistic.fit(x_train, y_train.flatten())

LogisticRegression()

In [41]:
y_pred = logistic.predict(x_test)

In [42]:
accuracy_score(y_test, y_pred)

0.995

In [43]:
knn = KNeighborsClassifier()
knn.fit(x_train, y_train.flatten())

KNeighborsClassifier()

In [44]:
y_pred_knn = knn.predict(x_test)

In [45]:
accuracy_score(y_test, y_pred_knn)

1.0

In [52]:
haardata = cv2.CascadeClassifier('../data.xml')

cap = cv2.VideoCapture(0)
# cap = cv2.VideoCapture('video_1.mp4')
faceData = []
font = cv2.FONT_HERSHEY_COMPLEX
while True:
    flag, img = cap.read()
    if flag:
        #img = cv2.resize(img, None, fx=0.5, fy=0.5)
        faces = haardata.detectMultiScale(img,1.2)
        for x,y,w,h in faces:
            cv2.rectangle(img, (x,y), (x+w, y+h), (0,0,255), 5)
            face = img[y:y+h, x:x+w, :]
            face = cv2.resize(face, (50,50))
            face = face / 255.0
            pred = logistic.predict(face.reshape(1,-1))
            name = names[int(pred)]
            #print(pred)
            cv2.putText(img, name, (x,y), font, 1, (0,255,0),1)
        cv2.imshow('img', img)
        if cv2.waitKey(1) == 27 or len(faceData) >= 400:
            break
    else:
        print("Camera is not detected...")
        break
cv2.destroyAllWindows()
cap.release()

In [50]:
from sklearn.metrics import confusion_matrix

In [51]:
confusion_matrix(y_test, y_pred)

array([[102,   1],
       [  0,  97]], dtype=int64)